In [1]:
import xarray as xr
import glob
import intake
import numpy as np
import seaborn as sns
from scipy import stats
import os

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
import xesmf as xe




In [3]:
from plotting_maps.acs_plotting_maps import plot_acs_hazard_multi, plot_acs_hazard, plot_data, cmap_dict, regions_dict
from matplotlib import colors, cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from dask.diagnostics import ProgressBar
# ProgressBar().register()

import dask
from dask.distributed import Client
client = Client(threads_per_worker=3, n_workers=14)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 42,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42077,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 42
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:33975,Total threads: 3
Dashboard: /proxy/42291/status,Memory: 8.94 GiB
Nanny: tcp://127.0.0.1:33575,


In [4]:
# for BARRA

In [5]:
# BARRA process has to be a bit different because the files are saved in single time slices, compared to single level with whole year


In [6]:
ds_target = xr.open_dataset("/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_6hr_1979.nc")
ds_target


<xarray.Dataset> Size: 4GB
Dimensions:    (time: 1460, lat: 436, lon: 777)
Coordinates:
  * time       (time) datetime64[ns] 12kB 1979-01-01 ... 1979-12-31T18:00:00
  * lon        (lon) float64 6kB 88.04 88.19 88.34 88.5 ... 207.6 207.8 207.9
  * lat        (lat) float64 3kB -53.58 -53.42 -53.27 ... 13.32 13.48 13.63
    pressure   int64 8B ...
Data variables:
    windspeed  (time, lat, lon) float64 4GB ...
Attributes:
    long_name:      Air Temperature
    standard_name:  air_temperature
    units:          K
    cell_methods:   pressure: point time: point (interval: 6H) area: interpol...
    grid_mapping:   crs

In [7]:
%%time
# testing
VARU = "wnd_ucmp"
VARV = "wnd_vcmp"

start_year = 1990
end_year = 2019

regridder=None
# target grid for regridding
ds_target = xr.open_dataset("/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_6hr_1979.nc")

def _preprocess(ds):
    try:
        ds = ds.drop_vars( [ "forecast_period", "forecast_reference_time", "latitude_longitude"],)\
            .sel({"pressure":250}, method = "nearest")\
            .sel(latitude= slice(-53.58 , 13.63), longitude=slice(88.04 , 207.9))
    except:
        ds = ds.drop_vars( ["latitude_longitude"],)\
            .sel({"pressure":250}, method = "nearest")\
            .sel(latitude= slice(-53.58 , 13.63), longitude=slice(88.04 , 207.9))
    return ds     

for year in np.arange(start_year, end_year+1):
    new_file = f"/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_{year}.nc"
    if os.path.exists(new_file):
        print(f"File '{new_file}' already exists.")
        continue
        
    # u component of wind
    print("filelist u...")
    filelist = glob.glob(f"/g/data/cj37/BARRA/BARRA_R/v1/analysis/prs/{VARU}/{year}/*/{VARU}-an-prs-PT0H-BARRA_R-v1-*T*00Z.sub.nc")
    filelist.sort()

    print("open_mfdataset u...")
    ds_u = xr.open_mfdataset(filelist, preprocess=_preprocess, combine="nested", concat_dim="time", parallel=True,)\
            .chunk({"time":-1, "latitude":220, "longitude":112})\
            .rename({f"{VARU}":"ua250", "latitude":"lat", "longitude":"lon"})

    # v component of wind
    print("filelist v...")
    filelist = glob.glob(f"/g/data/cj37/BARRA/BARRA_R/v1/analysis/prs/{VARV}/{year}/*/{VARV}-an-prs-PT0H-BARRA_R-v1-*T*00Z.sub.nc")
    filelist.sort()

    print("open_mfdataset v...")
    ds_v = xr.open_mfdataset(filelist, preprocess=_preprocess, combine="nested", concat_dim="time", parallel=True,)\
            .chunk({"time":-1, "latitude":220, "longitude":112})\
            .rename({f"{VARV}":"va250", "latitude":"lat", "longitude":"lon"})

    # calculate windspeed (lazy)
    print("calculate windspeed")
    ds_windspeed = xr.Dataset({"windspeed":xr.ufuncs.hypot(ds_u["ua250"], ds_v["va250"])}
                             ).chunk({"time":120, "lat":-1, "lon":-1})

    ds_u.close()
    ds_v.close()
    
    if regridder is None:
        print("calculate regridder...")
        regridder = xe.Regridder(ds_windspeed, ds_target, "bilinear", unmapped_to_nan=True)
    print(f"regrid and compute {year}...")
    ds_regridded = regridder(ds_windspeed).compute()
    try:
        ds_regridded.to_netcdf(new_file)
        print(f"Saved {new_file}")
    except:
        print(f"cannot save {new_file}")
    ds_windspeed.close()
    ds_regridded.close()
    

File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1990.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1991.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1992.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1993.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1994.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1995.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1996.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_wind

In [16]:
# # fix to replace zero values with nans
# for year in np.arange(start_year, end_year+1):
#     filename = f"/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_{year}.nc"
#     ds = xr.open_dataset(filename, use_cftime=True)
#     ds = ds.where(ds["windspeed"]!=0)
#     ds.to_netcdf(filename)
#     print(f"Saved {filename}")
# ds    